# [AI7008] AI실전연구프로젝트2
Professor: 김태영
<br><br>Student: Minseok Yang (msyang0809@khu.ac.kr)
<br>Number: 2023310384

---

# [Project] 낚시성 기사 탐지 모델

## Content

Data Preprocessing
<br/>Random Baseline 
<br/>Model Training
<br/>Model Inference
<br/>Clickbait Detection

---

### Data Preprocessing

In [ ]:
%pip install ipywidgets
%jupyter nbextension enable --py widgetsnbextension

In [1]:
from data_preprocessing import make_train_valid_json_xlsx_file_path_list
from data_preprocessing import write_jsontext_to_xlsx_file_with_batch_size
from data_preprocessing import write_xlsxtext_to_list_merge_file

In [49]:
json_path_list = ['data/aihub_clickbait/Training/Labeling'+ '/**/*.json', 
                  'data/aihub_clickbait/Validation/Labeling'+ '/**/*.json']
xlsx_path_list = ['data/aihub_clickbait/df/clickbait_classfication_train_', 
                 'data/aihub_clickbait/df/clickbait_classfication_valid_',
                 'data/aihub_clickbait/df/nonclickbait_classfication_train_', 
                 'data/aihub_clickbait/df/nonclickbait_classfication_valid_']

In [50]:
train_clickbait_json_file_path_list, train_nonclickbait_json_file_path_list, \
valid_clickbait_json_file_path_list, valid_nonclickbait_json_file_path_list, \
train_clickbait_xlsx_file_path_list, train_nonclickbait_xlsx_file_path_list, \
valid_clickbait_xlsx_file_path_list, valid_nonclickbait_xlsx_file_path_list = \
make_train_valid_json_xlsx_file_path_list(json_path_list, xlsx_path_list)

The number of train clickbait json file: 296035
The number of valid clickbait json file: 36991
The number of clickbait json file: 333026

The number of train nonclickbait json file: 290689
The number of valid nonclickbait json file: 36330
The number of nonclickbait json file: 327019

The number of train json file: 586724
The number of valid json file: 73321
The number of json file: 660045


In [51]:
batch_size = 1000
write_jsontext_to_xlsx_file_with_batch_size(train_clickbait_json_file_path_list, train_clickbait_xlsx_file_path_list, batch_size)

[Size]
The number of xlsx file: 296

[Order]


297it [54:54, 11.09s/it]                         


In [52]:
batch_size = 1000
write_jsontext_to_xlsx_file_with_batch_size(train_nonclickbait_json_file_path_list, train_nonclickbait_xlsx_file_path_list, batch_size)

[Size]
The number of xlsx file: 290

[Order]


291it [55:14, 11.39s/it]                         


In [53]:
batch_size = 1000
write_jsontext_to_xlsx_file_with_batch_size(valid_clickbait_json_file_path_list, valid_clickbait_xlsx_file_path_list, batch_size)

[Size]
The number of xlsx file: 36

[Order]


37it [06:46, 11.00s/it]                        


In [54]:
batch_size = 1000
write_jsontext_to_xlsx_file_with_batch_size(valid_nonclickbait_json_file_path_list, valid_nonclickbait_xlsx_file_path_list, batch_size)

[Size]
The number of xlsx file: 36

[Order]


37it [07:19, 11.87s/it]                        


In [3]:
xlsx_folder = './data/aihub_clickbait_detection/df/*.xlsx'
dataset_folder = './data/aihub_clickbait_detection/dataset/'
train_list, valid_list, test_list = write_xlsxtext_to_list_merge_file(xlsx_folder, dataset_folder)

The number of train text: 580856
The number of valid text: 73321
The number of test text: 5868


### Random Baseline 

In [1]:
!python -m score \
    --dataset_folder './data/aihub_clickbait_detection/dataset/' \
    --label [0, 1]

Accuracy: 49.51 %


### Model Training

In [ ]:
%pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
%pip install protobuf
%pip install --upgrade protobuf==3.20.1
%pip install openpyxl
%pip install --upgrade numpy==1.20.3

In [ ]:
import torch
from transformers import BertModel
from tokenization_kobert import KoBERTTokenizer

save_path = "./kobert"
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1')
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

In [3]:
import torch, gc, os
gc.collect()
torch.cuda.empty_cache()

In [15]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

!python -m train \
    --model_name 'klue/roberta-base' \
    --data_path './data/aihub_clickbait_detection/dataset/' \
    --train_dataset 'train_dataset.xlsx'  \
    --valid_dataset 'valid_dataset.xlsx' \
    --save_model_path './model/roberta_base/'  \
    --log_path './log/roberta_base/clickabait_detection_training_log.xlsx'  \
    --epoch 10  \
    --batch_size 64

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|                                         | 0/9076 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the lo

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

!python -m train \
    --model_name 'klue/roberta-base' \
    --data_path './data/aihub_clickbait_detection/dataset_preprocessing/' \
    --train_dataset 'train_dataset.xlsx'  \
    --valid_dataset 'valid_dataset.xlsx' \
    --save_model_path './model/roberta_base_preprocessing/'  \
    --log_path './log/roberta_base_preprocessing/clickabait_detection_training_log.xlsx'  \
    --epoch 10  \
    --batch_size 64

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|                                         | 0/9076 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the lo

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

!python -m train \
    --model_name 'klue/roberta-large' \
    --data_path './data/aihub_clickbait_detection/dataset/' \
    --train_dataset 'train_dataset.xlsx'  \
    --valid_dataset 'valid_dataset.xlsx' \
    --save_model_path './model/roberta_large/'  \
    --log_path './log/roberta_large/clickabait_detection_training_log.xlsx'  \
    --epoch 10  \
    --batch_size 16

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|                                        | 0/36304 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the l

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

!python -m train \
    --model_name 'klue/roberta-large' \
    --data_path './data/aihub_clickbait_detection/dataset_preprocessing/' \
    --train_dataset 'train_dataset.xlsx'  \
    --valid_dataset 'valid_dataset.xlsx' \
    --save_model_path './model/roberta_large_preprocessing/'  \
    --log_path './log/roberta_large_preprocessing/clickabait_detection_training_log.xlsx'  \
    --epoch 10  \
    --batch_size 16

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/opt/conda/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
Epoch 1:   0%|                                        | 0/36304 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the l

### Model Inference

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

!python -m inference \
    --model_name 'klue/roberta-base' \
    --data_path './data/aihub_clickbait_detection/dataset/' \
    --test_dataset 'test_dataset.xlsx'  \
    --load_model_path './model/roberta_base/'  \
    --log_path './log/roberta_base/clickabait_detection_evaluation_log.xlsx' \
    --inference_log_path './log/roberta_base/clickabait_detection_inference_log.xlsx' \
    --epoch 10  \
    --batch_size 1

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                  | 0/5868 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████████████████

In [6]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

!python -m inference \
    --model_name 'klue/roberta-base' \
    --data_path './data/aihub_clickbait_detection/dataset_preprocessing/' \
    --test_dataset 'test_dataset.xlsx'  \
    --load_model_path './model/roberta_base_preprocessing/'  \
    --log_path './log/roberta_base/clickabait_detection_evaluation_log.xlsx' \
    --inference_log_path './log/roberta_base_preprocessing/clickabait_detection_inference_log.xlsx' \
    --epoch 10  \
    --batch_size 1

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                  | 0/5868 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████████████████

In [7]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

!python -m inference \
    --model_name 'klue/roberta-large' \
    --data_path './data/aihub_clickbait_detection/dataset/' \
    --test_dataset 'test_dataset.xlsx'  \
    --load_model_path './model/roberta_large/'  \
    --log_path './log/roberta_large/clickabait_detection_evaluation_log.xlsx' \
    --inference_log_path './log/roberta_large/clickabait_detection_inference_log.xlsx' \
    --epoch 10  \
    --batch_size 1

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                  | 0/5868 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|█████████████████████

In [8]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

!python -m inference \
    --model_name 'klue/roberta-large' \
    --data_path './data/aihub_clickbait_detection/dataset_preprocessing/' \
    --test_dataset 'test_dataset.xlsx'  \
    --load_model_path './model/roberta_large_preprocessing/'  \
    --log_path './log/roberta_large/clickabait_detection_evaluation_log.xlsx' \
    --inference_log_path './log/roberta_large_preprocessing/clickabait_detection_inference_log.xlsx' \
    --epoch 10  \
    --batch_size 1

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.dense.weight', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                  | 0/5868 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|█████████████████████

### Clickbait Detection

In [17]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

!python -m clickbait_detection \
    --model_name 'klue/roberta-base' \
    --news_article_text_folder './data/aihub_clickbait_detection/news_article/' \
    --data_path './data/aihub_clickbait_detection/inference/' \
    --dataset 'dataset.xlsx'  \
    --label 1  \
    --load_model_path './model/roberta_base/'  \
    --clickbait_detection_log_path './log/roberta_base/clickabait_detection_log.xlsx' \
    --epoch 10  \
    --batch_size 1

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                     | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████████████████

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

!python -m clickbait_detection \
    --model_name 'klue/roberta-base' \
    --news_article_text_folder './data/aihub_clickbait_detection/news_article/' \
    --data_path './data/aihub_clickbait_detection/inference/' \
    --dataset 'dataset.xlsx'  \
    --label 1  \
    --load_model_path './model/roberta_base_preprocessing/'  \
    --clickbait_detection_log_path './log/roberta_base_preprocessing/clickabait_detection_log.xlsx' \
    --epoch 10  \
    --batch_size 1

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                     | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|██████████████████████

In [5]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

!python -m clickbait_detection \
    --model_name 'klue/roberta-large' \
    --news_article_text_folder './data/aihub_clickbait_detection/news_article/' \
    --data_path './data/aihub_clickbait_detection/inference/' \
    --dataset 'dataset.xlsx'  \
    --label 1  \
    --load_model_path './model/roberta_large/'  \
    --clickbait_detection_log_path './log/roberta_large/clickabait_detection_log.xlsx' \
    --epoch 10  \
    --batch_size 1

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|                                                     | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|█████████████████████████████████████████████| 1/1 [00:00<00:00,  1.90it/s]
Time 00:00:00    Accuracy: 1


In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

!python -m clickbait_detection \
    --model_name 'klue/roberta-large' \
    --news_article_text_folder './data/aihub_clickbait_detection/news_article/' \
    --data_path './data/aihub_clickbait_detection/inference/' \
    --dataset 'dataset.xlsx'  \
    --label 1  \
    --load_model_path './model/roberta_large_preprocessing/'  \
    --clickbait_detection_log_path './log/roberta_large_preprocessing/clickabait_detection_log.xlsx' \
    --epoch 10  \
    --batch_size 1

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.out_proj.weight', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|                                                     | 0/1 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2418: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
100%|█████████████████████

### Reference

<b>Data</b>
<br>[AI Hub 낚시성 기사 탐지 데이터](https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=71338)

<br><b>Paper</b>
<br>[Yinhan Liu et al. Robustly Optimized BERT Pretraining Approach. 2019.](https://aclanthology.org/N16-1174/)
<br>[Kevin Clark et al. ELECTRA: Pre-training Text Encoders as Discriminators Rather Than Generators. ICLR, 2020](https://arxiv.org/abs/2003.10555)
<br>[Zichao Yang et al. Hierarchical Attention Networks for Document Classification. NAACL, 2016.](https://aclanthology.org/N16-1174/)
<br>[Yongjie Wang et al. On the Use of Bert for Automated Essay Scoring: Joint Learning of Multi-Scale Essay Representation. NAACL, 2022.](https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11485497)
<br>[한상우, 온병원. 다중 계층 BERT를 활용한 낚시성 기사 탐지 모델. 한국정보기술학회, 2023.](https://www.dbpia.co.kr/journal/articleDetail?nodeId=NODE11485497)

<br><b>HuggingFace</b>
<br>[klue/roberta-base](https://huggingface.co/klue/roberta-base)
<br>[klue/roberta-large](https://huggingface.co/klue/roberta-large)

<br><b>Github</b>
<br>[hhk1364/Fishing-Article-Detection-Project-Backend](https://github.com/hhk1364/Fishing-Article-Detection-Project-Backend)
<br>[SKTBrain/KoBERT](https://github.com/SKTBrain/KoBERT)
<br>[monologg/KoBERT-Transformers](https://github.com/monologg/KoBERT-Transformers)
<br>[monologg/KoELECTRA](https://github.com/monologg/KoELECTRA)
<br>[jaehyeongAN/KoELECTRA-finetuned-sentiment-analysis](https://github.com/jaehyeongAN/KoELECTRA-finetuned-sentiment-analysis)
<br>[lkkaram/korean-frown-sentence-classifier](https://github.com/lkkaram/korean-frown-sentence-classifier)
<br>[kimwoonggon/publicservant_AI](https://github.com/kimwoonggon/publicservant_AI/)
<br>[lingochamp/Multi-Scale-BERT-AES](https://github.com/lingochamp/Multi-Scale-BERT-AES/)
<br>[Doheon/NewsClassification-KoBERT](https://github.com/Doheon/NewsClassification-KoBERT)
<br>[jaylnne/nsmc-bert-pytorch_lightning](https://github.com/jaylnne/nsmc-bert-pytorch_lightning)

<br><b>Blog</b>
<br>[Week 28 - BERT만 잘 써먹어도 최고가 될 수 있다?](https://jiho-ml.com/weekly-nlp-28/)
<br>[Week 37 - NLP 모델, 낚시성 기사 방지 효과 검증돼](https://jiho-ml.com/weekly-nlp-37/)

<br><b>Dacon</b>
<br>[[private 3rd] klue/roberta + teacher-student](https://dacon.io/en/competitions/official/235747/codeshare/3072)
